In [ ]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image, ImageOps
import requests
import torch
import matplotlib.pyplot as plt
import numpy as np
import json
import torch
import requests
from tqdm import tqdm
from io import BytesIO
from diffusersgrad import StableDiffusionImg2ImgPipeline
import torchvision.transforms as T
from torchmetrics.multimodal import *
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.inception import InceptionScore
from torchmetrics.image import PeakSignalNoiseRatio
from torchmetrics.image import MultiScaleStructuralSimilarityIndexMeasure
from torchmetrics.image import StructuralSimilarityIndexMeasure
import torchvision.transforms as transforms


from utils import preprocess, recover_image
to_pil = T.ToPILImage()

with open("dataset/prompts.json", "r") as f:
    prompts_dict = json.load(f)
image_names = list(prompts_dict.keys())

image_idxs = [0,2,7,8,16,17,19,20,23,25,26,27,31,33,35,36,41,42,43,44,45,46,49,50,51,52,54,55,56,58,61,63,67,71,
77,78,82,83,85,87,88,89,91,92,97,98,99,100,116,118,129,131,133,134,137,138,148,155,157,159,162,166,167,172,
178,182,183,184,186,189,192,193,195,197,199,201,206,207,208,209,212,213,214,215,216,217,231,235,237,239,240,
241,250,251,252,254,255,256,266,268,269,271,272,279,285,286,287,288,293,299,300,301,310,321,326,327,331,332,
333,338,339,343,345,349,351,355,356,361,363,364,367,369,370,375,377,378,380,382,383,387,390,391,397]


# A fixed random selected seed in all the experiments
SEED = 9209
torch.manual_seed(SEED)

torch.cuda.empty_cache()

transform = transforms.Compose([
    transforms.PILToTensor()
])

metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")
fid = FrechetInceptionDistance()
inception = InceptionScore()
psnr = PeakSignalNoiseRatio(data_range=1.0)
ms_ssim = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0)
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)

In [ ]:
# Generated images by adversarial samples
folder = "data/inpainting/encoder_generate/"
# Generated images by original samples
folder_ori_generate = "data/inpainting/ori_images_generate/"
# Original images
folder_ori = "dataset/images_crop/"

In [ ]:
#FID
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori + image_names[image_idx]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.uint8)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.uint8)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
fid.update(image_advs, real=False)
fid.update(image_oris, real=True)
torch.manual_seed(SEED)
score = fid.compute()
score = score.detach().cpu().numpy()
print("FID: ", score)

In [ ]:
#IS
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori + image_names[image_idx]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.uint8)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.uint8)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
inception.update(image_advs)
torch.manual_seed(SEED)
score = inception.compute()
IS = score[0].detach().cpu().numpy()
print("IS: ", IS)

In [ ]:
#PSNR
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori_generate + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.float32)/255.0
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.float32)/255.0
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
torch.manual_seed(SEED)
score = psnr(image_advs, image_oris)

print("PSNR: ", score)

In [ ]:
#SSIM
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori_generate + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.float32)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.float32)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
torch.manual_seed(SEED)
score = ssim(image_advs, image_oris)

print("SSIM: ", score)

In [ ]:
#MSSSIM
msssim = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0)
total = 0
scores = 0.0
num = 100
image_advs = []
image_oris = []
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    for j in range(5):
        adv = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        ori = Image.open(folder_ori_generate + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        torch.manual_seed(SEED)
        adv = np.array(adv).astype(np.float32)
        adv = adv[None].transpose(0, 3, 1, 2)
        image_adv = torch.from_numpy(adv)
        image_advs.append(image_adv)
        ori = np.array(ori).astype(np.float32)
        ori = ori[None].transpose(0, 3, 1, 2)
        image_ori = torch.from_numpy(ori)
        image_oris.append(image_ori)

image_advs = torch.concat(image_advs, dim=0)
image_oris = torch.concat(image_oris, dim=0)
print(image_advs.shape)
torch.manual_seed(SEED)
score = msssim(image_advs, image_oris)

print("MSSSIM: ", score)

In [ ]:
#CLIP Score
total = 0
scores = 0.0
num = 100
for i in range(num):
    image_idx = image_idxs[i]
    torch.cuda.empty_cache()
    if total % 20 == 0:
        print(total)
    prompts = prompts_dict[image_names[image_idx]]
    for j in range(5):
        image = Image.open(folder + image_names[image_idx][:-4] + "_" + str(j) + image_names[image_idx][-4:]).convert('RGB').resize((512,512))
        prompt = prompts[j]
        torch.manual_seed(SEED)
        score = metric(transform(image), prompt)
        score = score.detach().cpu().numpy()
        scores += score
    total += 1
print(scores/(num*5))